In [0]:
#ANTONIO este lo tuve que hacer uno por uno porque me daba fallo
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# univariate multi-step encoder-decoder cnn-lstm
from math import sqrt
import numpy as np
from numpy import array
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Embedding
from keras import backend as K
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization, CuDNNGRU, CuDNNLSTM, ConvLSTM2D
from keras.layers import Flatten, TimeDistributed, Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.regularizers import l1_l2, l1, l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
def split_dataset(df, param):
  #scaler = MinMaxScaler(feature_range=(-1,1))
 
  grouped = df.groupby(['measure','capture'])
  
  n_features=2 #era 3
  n1 = 2048
  n2 = 1024
  len_forecast = param

  step = 8
  split = 0.8
  
  np.random.seed(42)
  
  X_train = []
  X_test = []
  y_train = []
  y_test = []
  for name, group in grouped:
    arr = group.values
    for i in range(0,len(arr)-n1,n1):
        a = arr[i:i+n1]
        option = np.random.rand()
        for j in range(0,len(a)-n2,step):
            b = a[j:j+n2]
            # Case NO stalling event:
            if option <= split:
                X_train.extend(b[:-len_forecast,:-3]) #:-3 en esta y las 3 d abajo
                y_train.extend(b[-len_forecast:,:-3])
            else:
                X_test.extend(b[:-len_forecast,:-3])
                y_test.extend(b[-len_forecast:,:-3])
  #%%
  #X_train = [item for sublist in X_train for item in sublist]
  #y_train = [item for sublist in y_train for item in sublist]

  #X_test = [item for sublist in X_test for item in sublist]
  #y_test = [item for sublist in y_test for item in sublist]
  
  #scaler.fit(X_train)
  #X_train=scaler.transform(X_train)
  #%%
  # reshape for keras (samples, timestep, features)
  X_train = np.reshape(X_train, (-1, n2-len_forecast, n_features))
  y_train = np.reshape(y_train, (-1, len_forecast, n_features))

  X_test = np.reshape(X_test, (-1, n2-len_forecast, n_features))
  y_test = np.reshape(y_test, (-1, len_forecast, n_features))

  #print(y_train.shape)
  #print(y_test[0])
  #print(y_test.shape)

  #%%
  norm = X_train.max(axis=0)[0] 
  X_train = (X_train / norm) #<- eso -1 pa normalizar entre -1 y 1
  y_train = (y_train / norm)

  X_test = (X_test / norm) 
  y_test = (y_test / norm) 

  return X_train, y_train, X_test, y_test

In [0]:
def build_model(train_x, train_y, test_x, test_y):
  # prepare data
  #train_x, train_y = to_supervised(train, n_input)
  #train_x shape(1093, 14, 1) <- inputs
  #train_y shape(1093, 7)
  # define parameters
  #train_x = np.array(train_x)
  #train_y = np.array(train_y)
  verbose, epochs, batch_size = 1, 10, 64
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  
  # reshape output into [samples, timesteps, features]
 # train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
  # define model
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#  model.add(Conv1D(filters=50, kernel_size=7, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(RepeatVector(n_outputs))
  
  model.add(Dropout(0.5))
  model.add(CuDNNLSTM(200, return_sequences=True))
#  model.add(CuDNNLSTM(50, return_sequences=True))
#  model.add(CuDNNLSTM(10,return_sequences=True))

  model.add(TimeDistributed(Dense(100, activation='relu')))
#  model.add(TimeDistributed(Dropout(0.5)))
#  model.add(TimeDistributed(Dense(5, activation='sigmoid')))
  #model.add(TimeDistributed(Dropout(0.2)))
  model.add(TimeDistributed(Dense(n_features)))
  
  
  model.compile(loss='mse', optimizer='adam')
  # fit network
  history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose,
           validation_data=(test_x, test_y))
  return model, history

In [0]:
def make_prediction(model, X_test, y_test, n_pred):
  yhat = model.predict(X_test, verbose=1)
  a=y_test.reshape(y_test.shape[0]*y_test.shape[2], y_test.shape[1])
  b=yhat.reshape(yhat.shape[0]*yhat.shape[2], yhat.shape[1])
  c=X_test.reshape(X_test.shape[0]*X_test.shape[2], X_test.shape[1])
  n_vent=1024
  #n_pred=80
  n=2

  plt.plot(a[(n*n_vent)-n_pred:(n*n_vent), 0], 'g', marker='.')
  plt.plot(b[(n*n_vent)-n_pred:(n*n_vent), 0], 'r', marker='.')
  

In [0]:
df = pd.read_csv('./gdrive/My Drive/Colab Notebooks/dataset.csv')
df2 = df[["pkt_length", "tcp_window_size_value"]].rolling(10).mean()
#df2.insert(3, "stalling_event")
df2['stalling_event'] = df.stalling_event.values
df2['measure'] = df.measure.values
df2['capture'] = df.capture.values

In [0]:
df3 = df2.dropna()


In [0]:
values = [100] #32 64 128 256
all_test = list()
splits = 8 #era 16

#for param in values:
X_train, y_train, X_test, y_test = split_dataset(df3, 100)

model, hist = build_model(X_train, y_train, X_test, y_test)#, n_length, n_features, es, checkpoint, reduce_lr) 

val_loss = hist.history['val_loss']

all_test.append(val_loss[-1])
print(all_test)
make_prediction(model, X_test, y_test, 100)


Train on 35712 samples, validate on 8704 samples
Epoch 1/10


InternalError: ignored

In [0]:
plt.plot(values, all_test, label='test', marker='o')
#pyplot.legend()
plt.show()